# Model Training

In this notebook, we will ask you a series of questions regarding model selection. Based on your responses, we will ask you to create the ML models that you've chosen. 

The bonus step is completely optional, but if you provide a sufficient third machine learning model in this project, we will add `1000` points to your Kahoot leaderboard score.

**Note**: Use the dataset that you've created in your previous data transformation step (not the original model).

## Questions
Is this a classification or regression task?  

Answer here

- This is a Classification Task because our goal here is to predict whether a transaction is fraudulent or not.

Are you predicting for multiple classes or binary classes?  

- Binary, because our goal is to assign each transaction to one of two categories: Fraudulent or Not Fraudulent.

Given these observations, which 2 (or possibly 3) machine learning models will you choose?  

- Logistic Regression: simple, fast, and interpretable baseline for binary classification.
- Random Forest Classifier: a powerful ensemble model that handles nonlinear relationships well.
- XGBoost (Extreme Gradient Boosting): highly accurate and efficient gradient boosting algorithm.
- Use all of the above with Stratified Cross-Validation and SMOTE. For imbalanced data like fraud detection, these help improve model performance on the minority class.

## First Model

Using the first model that you've chosen, implement the following steps.

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.linear_model import Lasso, Ridge

### 1) Create a train-test split

Use your cleaned and transformed dataset to divide your features and labels into training and testing sets. Make sure you’re only using numeric or properly encoded features.  

In [34]:
dummy_transactions = pd.read_csv("dummy_transactions.csv")
dummy_transactions

,amount,oldbalanceOrg,newbalanceOrig,isFraud,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,983.09,36730.24,35747.15,0,0,False,False,False,True,False
1,55215.25,99414.00,44198.75,0,0,False,False,False,True,False
2,220986.01,7773074.97,7994060.98,0,0,True,False,False,False,False
3,2357394.75,0.00,0.00,0,0,False,False,False,False,True
4,67990.14,0.00,0.00,0,0,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...
999995,13606.07,114122.11,100516.04,0,0,False,False,False,True,False
999996,9139.61,0.00,0.00,0,0,False,False,False,True,False
999997,153650.41,50677.00,0.00,0,0,False,True,False,False,False
999998,163810.52,0.00,0.00,0,0,False,True,False,False,False


In [ ]:
from sklearn.model_selection import train_test_split

# dropping isFraud and setting isFraud as target
X = dummy_transactions.drop(columns='isFraud')
y = dummy_transactions['isFraud']

#  train-test split with stratification to keep class proportions
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y  
)

### 2) Search for best hyperparameters
Use tools like GridSearchCV, RandomizedSearchCV, or model-specific tuning functions to find the best hyperparameters for your first model.

In [38]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

In [ ]:
# sample 10% of the training data for quicker tuning
X_sample = X_train.sample(frac=0.1, random_state=42)
y_sample = y_train.loc[X_sample.index]


In [ ]:
# RandomForest
rf = RandomForestClassifier(random_state=42)

# define the hyperparameter distributions
param_dist = {
    'n_estimators': randint(100, 300),
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

# Randomized Search to save time processing a large dataset
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=5,             # Try 5 random combinations
    cv=2,                  # Use 2 folds to make it faster
    scoring='f1',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

random_search.fit(X_sample, y_sample)

print("Best Hyperparameters:", random_search.best_params_)
best_rf = random_search.best_estimator_

Fitting 2 folds for each of 5 candidates, totalling 10 fits
Best Hyperparameters: {'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 206}


### 3) Train your model
Select the model with best hyperparameters and generate predictions on your test set. Evaluate your models accuracy, precision, recall, and sensitivity.  

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# adding the best hyperparameters we got from the previous process
best_rf = RandomForestClassifier(
    max_depth=20,
    max_features='log2',
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=206,
    random_state=42
)

# train the model and make predictions
best_rf.fit(X_train, y_train)
y_pred = best_rf.predict(X_test)

# evaluate 
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")

Accuracy:  0.9991
Precision: 1.0000
Recall:    0.3158
F1 Score:  0.4800


## Second Model

Create a second machine learning object and rerun steps (2) & (3) on this model. Compare accuracy metrics between these two models. Which handles the class imbalance more effectively?

Create as many code-blocks as needed.

In [49]:
from imblearn.over_sampling import SMOTE

# another way to sample data
sample_data = dummy_transactions.sample(n=50000, random_state=42)

# split features and target
X = sample_data.drop('isFraud', axis=1)
y = sample_data['isFraud']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# resample with SMOTE 
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
# logistic regression model
log_reg = LogisticRegression(solver='liblinear', random_state=42)

# hyperparameter grid
param_dist = {
    'C': np.logspace(-3, 2, 10),  # e.g. 0.001 to 100
    'penalty': ['l1', 'l2']
}

# random search
random_search = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=param_dist,
    n_iter=10,
    scoring='f1',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# fit on resampled training data
random_search.fit(X_train_resampled, y_train_resampled)

# output best model
best_logreg = random_search.best_estimator_
print("Best Hyperparameters:", random_search.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters: {'penalty': 'l1', 'C': np.float64(0.001)}


c:\Users\nadya\miniconda3\envs\ds\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [52]:
# adding the best hyperparameters we got from the previous process
best_logreg = LogisticRegression(
    solver='liblinear',   # 'liblinear' supports 'l1' penalty
    penalty='l1',
    C=0.001,
    random_state=42
)

best_logreg.fit(X_train_resampled, y_train_resampled)

# predict 
y_pred = best_logreg.predict(X_test)

# evaluate 
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:   ", recall_score(y_test, y_pred))
print("F1 Score: ", f1_score(y_test, y_pred))

Accuracy:  0.9641333333333333
Precision: 0.032432432432432434
Recall:    0.9473684210526315
F1 Score:  0.0627177700348432


c:\Users\nadya\miniconda3\envs\ds\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### (Bonus/Optional) Third Model

Create a third machine learning model and rerun steps (2) & (3) on this model. Which model has the best predictive capabilities? 

Create as many code-blocks as needed.